In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!git clone 'https://github.com/Beomi/KoAlpaca.git'
!pip install -r /content/KoAlpaca/requirements.txt
!pip install -U torch transformers tokenizers accelerate

Cloning into 'KoAlpaca'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 248 (delta 84), reused 102 (delta 70), pack-reused 126
Receiving objects: 100% (248/248), 31.60 MiB | 22.17 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
    

## 테스트 데이터
- KoAlpaca dataset v1.1(30개, 지식 in 질답)
- KoAlpaca dataset v1.0 (50개, alpaca dataset 한글 번역본)
- 선별한 Koalpaca dataset v1.0(14개) + shareGPT 번역본(6개)
  - 수학 문제, 감정 분석, 요약, 문장 변환 등

KoAlpaca dataset v1.1 (30개)

In [3]:
import json
import pandas as pd

file_path = '/content/KoAlpaca/KoAlpaca_v1.1.jsonl'
dataset = []

with open(file_path, 'r', encoding = 'utf-8') as f:
  for line in f:
    line = line.strip()
    if line:
      dataset.append(json.loads(line))

test_df = pd.DataFrame(dataset[:30])
test_df = test_df.drop('url', axis = 1)
test_df['input'] = ""
test_df = test_df.reindex(columns = ['instruction', 'input', 'output'])
test_df

,instruction,input,output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...
1,스웨터의 유래는 어디에서 시작되었나요?,,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카..."
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...",,"'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로..."
5,씨앗을 냉장 처리후 심는 이유와 장기보존하려면 어떻게 해야 할까요?,,1. 씨앗을 발아과정에서 냉장실에 보관후 심는 이유는 씨앗에는 휴면기간이 있기 때문...
6,야구 글러브의 종류와 포지션별 특징은 무엇인가요?,,"보통 4칸으로 그물처럼 되어있는 것이 내야글러브이며, 1루를 제외한 내야 전 포지션..."
7,OPEC+와 OPEC의 차이점은 무엇인가요?,,석유 수출 기구(OPEC)와 러시아 등 10개 주요 산유국의 연합체(OPEC+)가 ...
8,광해군은 폭군이었나요? 그 업적은 어땠나요?,,"광해군이 폭군인가에 대해서는 서인, 인조, 인목대비 등의 시각에서는 폭군이라고 말하..."
9,안과에서 눈치료할 때 쓰는 빨간 불은 무엇인가요? 그리고 눈을 감고 있는 이유는 무...,,"안과에서 눈을 치료할 때 사용하는 빨간 불은 적외선입니다. 이때, 적외선은 눈을 감..."


KoAlpaca dataset v1.0 (50개)

In [4]:
file_path = '/content/KoAlpaca/ko_alpaca_data.json'
dataset2 = []

with open(file_path, 'r', encoding = 'utf-8') as f:
  json_data = json.load(f)

test_df2 = pd.DataFrame(json_data[:50])
test_df2

,instruction,input,output
0,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는..."
1,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다."
2,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있..."
3,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기..."
4,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....
5,홀수 중 하나를 밝히세요.,"트위터, 인스타그램, 텔레그램",텔레그램입니다.
6,다음 분수가 1/4에 해당하는 이유를 설명하세요.,4/16,분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.
7,중요한 진로 결정을 내려야 하는 주인공의 이야기를 3인칭 내레이션으로 단편 소설을 ...,,"안나는 여러 대학교의 입학 가능성이 있었으나, 제발스러운 직업들을 향해 유혹 당하는..."
8,집의 3D 모델을 렌더링합니다.,,집의 3D 모델링 결과물 입니다. (이미지 첨부)
9,이 문장에 철자와 문법 오류가 있는지 평가하세요.,그는 식사를 마치고 식당을 나섰습니다.,그 문장에는 철자나 문법에 대한 오류가 없습니다.


선별한 Koalpaca dataset v1.0(14개) + shareGPT 번역본(6개)

In [5]:
test_data_input = [
      {
        "instruction":"다음 문장을 참 또는 거짓으로 분류하십시오.",
        "input":"대법원은 미국 최고법원입니다.",
        "output":"참."
      },
      {
        "instruction":"아래 문장의 각 단어에 품사를 지정하십시오.",
        "input":"새끼 고양이는 종종 신나게 뛰어다니기도 합니다.",
        "output":"새끼(Noun) 고양이(Noun) 는 은 종종 Adverb 신나게 Adverb 뛰어다니기도 합니다."
      },
      {
        "instruction":"이 텍스트의 주요 아이디어는 무엇인가요?",
        "input":"로봇 산업은 오늘날 경제에서 가장 빠르게 성장하는 분야 중 하나가 되었습니다. 로보틱스 기술은 제조, 의료, 국방 등 다양한 산업과 애플리케이션에 적용되고 있습니다.",
        "output":"로봇 공학은 빠르게 성장하는 분야이며, 제조, 의료, 국방 등 다양한 산업과 애플리케이션에 적용됩니다."
      },
      {
        "instruction":"화씨에서 섭씨로 온도를 변환합니다.",
        "input":"94F",
        "output":"입력받은 화씨 94도는 섭씨 34.44˚입니다."
      },
      {
        "instruction":"다음 문장을 단수형에서 복수형으로 바꾸세요.",
        "input":"고양이가 술을 마시고 있습니다.",
        "output":"고양이들이 술을 마시고 있습니다."
      },
      {
        "instruction":"다음 기사를 자신의 말로 요약하세요.",
        "input":"인터넷은 현대 생활의 중요한 일부가 되었으며, 우리가 소통하고 비즈니스를 수행하는 방식에 혁명을 일으켰습니다. 인터넷은 아이디어와 경험을 신속하고 저렴하게 전 세계로 확산할 수 있게 해 주었습니다.",
        "output":"인터넷은 우리가 개인적으로 교류하고 국제 비즈니스를 수행하는 방식을 혁신시킨 중요한 부분입니다."
      },
      {
        "instruction":"이 문장을 미래 시제로 바꾸세요.",
        "input":"그녀는 걸어서 학교에 다닙니다.",
        "output":"그녀는 걸어서 학교에 갈 것입니다."
      },
      {
        "instruction":"이차 방정식의 해를 구하세요.",
        "input":"3x² + 7x - 4 = 0",
        "output":"3x² + 7x - 4 = 0을 풀면 x = -1 or 4\/3입니다."
      },
      {
        "instruction":"다음 역할에 대한 직무 설명 작성",
        "input":"데이터 분석 컨설턴트",
        "output":"데이터 분석 컨설턴트는 회사에서 데이터에 대한 전문 지식이 필요한 일들을 수행하는 사람입니다. 이 분야는 데이터 수집, 분석, 모델링 그리고 데이터 분석 도구 소프트웨어를 사용합니다."
      },
      {
        "instruction":"주어진 문장을 좀 더 격식 있는 어조로 다시 써보세요.",
        "input":"급하게 가게에 가야 했어요.",
        "output":"급한 일이 발생하여 거센 속도로 상점을 향했습니다."
      },  
      {
        "instruction":"다음 텍스트를 긍정 또는 부정의 두 가지 감정 클래스 중 하나로 분류합니다.",
        "input":"너무 느리고 사용자 친화적이지 않아서 제품이 그다지 마음에 들지 않았습니다.",
        "output":"부정적인 감정 클래스입니다."
      },
      {
        "instruction":"단락을 문장으로 요약합니다.",
        "input":"가구를 구매할 때는 공간의 전체적인 모양과 느낌을 고려하는 것이 중요합니다. 즉, 선택하려는 가구가 기존 가구와 잘 어울리는지, 전체적인 스타일은 어떤 모습일지 생각해봐야 합니다.",
        "output":"맞춤형 가구 구매를 위해서는, 전체적인 공간 느낌과 스타일을 고려해야 합니다."
      },
      {
        "instruction":"다음 문장을 예\/아니오 질문으로 바꿉니다.",
        "input":"내일 비가 오겠습니다.",
        "output":"내일 비가 오겠습니까?"
      },
      {
        "instruction":"주어진 기사를 긍정적 또는 부정적 감정으로 분류하세요.",
        "input":"새 차는 실망스럽습니다. 브레이크가 끔찍하고 제공되는 기능에 비해 비용이 너무 많이 듭니다.",
        "output":"기사 분류 결과, \"새 차는 실망스럽습니다.\" 문장은 부정적인 감정을 포함하고 있습니다."
      },
      {
        "instruction":"언어 학습에 대한 도그메 접근법과 언어 학습에 대한 어휘 접근법의 차이점은 무엇인가요?",
        "input":"",
        "output":"도그미 접근법과 어휘 접근법은 외국어를 가르치는 데 있어 서로 다른 두 가지 방법론입니다.\r\n\r\n도그메 접근법은 도그메 ELT(영어 교육)라고도 하며, 교사가 주도하는 학습자 중심의 방법론으로, 새로운 언어와 학습자의 필요와 관심사에 초점을 맞춥니다. 교재에 의존하지 않고, 대화 활동에 중점을 두며, 학습자가 자발적으로 만들어내는 언어를 강조하는 것이 특징입니다. Dogme의 목표는 학생들의 개인적인 경험과 동기를 활용하여 학생들에게 매력적이고 의미 있는 학습 경험을 제공하는 것입니다.\r\n\r\n반면 어휘 접근법은 외국어 습득의 주요 수단으로서 어휘 습득에 초점을 맞춘 교수법입니다. 어휘 접근법은 효과적인 의사소통을 위해 목표 언어의 어휘를 많이 습득하는 것이 중요하다는 점을 강조합니다. 문법 규칙과 구조는 문맥에서 단어에 노출되면 자연스럽게 따라온다고 주장합니다. 어휘 접근법은 의미가 언어의 중심이며, 따라서 어휘는 의미 있는 덩어리 또는 \"어휘 구문\"으로 가르쳐야 한다는 생각에 기초합니다.\r\n\r\n요약하자면, 도그메 접근법은 학습자의 자발적인 언어와 개인적인 경험을 강조하는 반면, 어휘 접근법은 어휘 습득을 언어 학습의 핵심으로 강조합니다."
      },
      {
        "instruction":"우주왕복선에 있는 반응 제어 시스템의 내결함성을 설명하십시오.",
        "input":"",
        "output":"우주왕복선의 반응 제어 시스템(RCS)은 내결함성으로 설계되었기 때문에 구성 요소 중 하나 이상이 고장 나더라도 계속 작동할 수 있었습니다. RCS는 1차 추진기와 2차 추진기 두 세트로 구성되었습니다. 1차 추진기는 궤도선의 후미에 위치했으며 궤도 진입 및 탈궤도와 같은 주요 기동에 사용되었습니다. 보조 추진기는 궤도선의 기수와 꼬리에 위치하며 우주선의 자세 또는 우주에서의 방향을 미세 조정하는 데 사용되었습니다.\r\n\r\n각 추진기 세트에는 여러 개의 예비 추진기가 있었기 때문에 한 개가 고장 나더라도 다른 추진기가 계속 작동할 수 있었습니다. 또한 RCS는 궤도선의 메인 컴퓨터에 연결되어 시스템 성능을 모니터링하고 필요 시 자동으로 백업 추진기로 전환할 수 있었습니다. 이를 통해 추진기가 고장 나더라도 우주선을 계속 제어할 수 있었습니다.\r\n\r\n이러한 모든 기능 덕분에 RCS는 내결함성을 갖추고 고장이 발생하더라도 계속 작동할 수 있었으며, 이는 우주왕복선의 중요한 안전 기능으로 자리 잡았습니다."
      },
      {
        "instruction":"r미켈 셀코에 대한 랩 노래 쓰기",
        "input":"",
        "output":"1절\r\n여러분, 잘 들어요, 할 얘기가 있어요.\r\n미켈 셀코라는 남자의 이야기입니다. 그는 잘하고 있습니다.\r\n노르웨이의 거리에서 그는 정상에 올랐어\r\n너무 차가워서 심장이 멈출 정도야\r\n그는 며칠 동안 바를 가지고 있고 자신 만의 스타일이 있습니다.\r\n그가 한 구절을 떨어 뜨릴 때 군중은 신음하기 시작합니다.\r\n그는 가사에 천재적인 재능을 가졌지\r\n마이크에 올라서면 불사조 같아\r\n\r\n코러스\r\n미켈 셀코, 미켈 셀코\r\n그는 게임의 왕, 그는 흐름을 가지고 있습니다.\r\n미켈 셀코, 미켈 셀코\r\n그는 진짜 거래, 당신은 이미 알고 있습니다.\r\n\r\n2절\r\n그는 라임을 너무 달콤하게 만드는 방법을 가지고있어.\r\n그의 가사는 꿀 같아서 이길 수 없어\r\n그는 공유할 메시지와 이야기가 있어\r\n그리고 그가 마이크 위에있을 때 그는 소원 우물과 같습니다.\r\n그는 황금의 심장과 불의 정신을 가졌어\r\n그리고 그가 바를 뱉을 때 그는 진정한 욕망입니다.\r\n그는 매일 성장하는 팬층을 가지고 있습니다.\r\n그가 무대에 오르면 그들은 환호성을 지르죠\r\n\r\n코러스\r\n미켈 셀코, 미켈 셀코\r\n그는 게임의 왕이야, 그는 흐름이 있어.\r\n미켈 셀코, 미켈 셀코\r\n그는 진짜야, 이미 알잖아\r\n\r\nOutro:\r\n아직 미켈 셀코에 대해 들어보지 못했다면\r\n잘 들어요, 그가 위협을 가하고 있으니까요\r\n그는 게임을 장악하고 자신의 것으로 만들고 있습니다.\r\n그리고 그가 끝내면, 아무도 그를 쫓아낼 수 없을거야.\r\n미켈 셀코를 주목해야 해요\r\n그는 흐름이 있고 그것을 뜨겁게 만들고 있습니다.\r\n\r\n참고: 이 노래는 가상의 노래로, 미켈 셀코에 대해 전혀 알지 못한 채 지어낸 곡입니다."
      },
      {
        "instruction":"최고의 아웃도어 박람회는 무엇인가요?",
        "input":"",
        "output":"세계 최고의 아웃도어 박람회에는 다음과 같은 것들이 있습니다:\r\n\r\n1. 아웃도어 리테일러(OR) - 미국에서 개최되는 이 박람회는 아웃도어 업계에서 가장 규모가 크고 잘 알려진 박람회 중 하나로, 다양한 아웃도어 장비와 의류를 선보입니다.\r\n2. ISPO - 독일에서 개최되는 이 박람회는 스포츠 업계에서 가장 큰 국제 무역 박람회 중 하나로, 아웃도어 및 겨울 스포츠의 최신 제품과 트렌드를 선보입니다.\r\n3. 아웃도어 바이 ISPO - 독일에서 개최되는 이 박람회는 아웃도어 장비 및 의류에 중점을 두고 있으며, 유럽 아웃도어 시장에서 가장 중요한 행사 중 하나로 꼽힙니다.\r\n4. SIA 스노우 쇼 - 미국에서 개최되는 이 전시회는 겨울 스포츠 산업에 초점을 맞춘 전시회로, 스노우 스포츠 장비 및 의류의 최신 제품과 트렌드를 선보입니다.\r\n5. 아웃도어 & 바이크 - 독일에서 개최되는 이 무역 박람회는 아웃도어 및 사이클링 제품에 중점을 두며, 두 산업의 최신 장비와 트렌드를 소개합니다.\r\n6. 아웃도어 프리드리히스하펜 - 독일에서 개최되는 이 무역 박람회는 아웃도어 장비 및 의류에 중점을 둡니다.\r\n7. 차이나 아웃도어 - 중국에서 개최되는 이 무역 박람회는 아웃도어 장비, 의류 및 장비에 중점을 두고 있으며 아시아에서 가장 중요한 아웃도어 행사 중 하나로 꼽힙니다.\r\n8. 아웃도어 쇼 - 영국에서 개최되는 이 무역 박람회는 아웃도어 장비, 의류 및 장비에 중점을 둡니다.\r\n\r\n이 전시회는 유명하고 평판이 좋은 전시회 중 일부이며, 전 세계 곳곳에서 아웃도어 산업의 특정 요구 사항을 충족하는 다른 많은 전시회가 열립니다."
      },
      {
        "instruction":"메타버스 설명",
        "input":"",
        "output":"메타버스는 모든 가상 세계, 증강 현실 및 인터넷의 합계를 포함하여 가상으로 향상된 물리적 현실과 물리적으로 지속되는 가상 공간이 융합되어 생성된 집단적 가상 공유 공간을 설명하는 데 사용되는 용어입니다. 가상 세계, 증강 현실, 인터넷 등 모든 가상 세계의 합을 포함하여 가상으로 강화된 물리적 현실과 물리적으로 지속되는 가상 공간이 융합되어 만들어진 집합적인 가상 공유 공간입니다. 이 용어는 공상과학 소설가 닐 스티븐슨이 1992년 발표한 소설 '스노우 크래시'에서 사람들이 서로, 그리고 가상의 사물 및 환경과 상호작용할 수 있는 가상 현실 세계를 묘사하면서 만들어졌습니다.\r\n\r\n메타버스에서 사람들은 여러 사용자가 공유하는 가상 세계에서 서로, 그리고 가상 객체 및 환경과 상호 작용할 수 있습니다. 이 가상 세계는 실제 위치를 기반으로 할 수도 있고 완전히 허구일 수도 있습니다. 사람들은 아바타 또는 자신의 가상 표현을 사용하여 서로 및 가상 세계와 상호 작용할 수 있습니다.\r\n\r\n메타버스라는 개념은 최근 몇 년 동안 가상 현실과 증강 현실 기술의 발달로 사람들이 몰입감 있는 가상 환경을 경험할 수 있게 되면서 인기를 얻고 있습니다. 또한 메타버스는 교육, 엔터테인먼트, 커머스 등 다양한 애플리케이션을 위한 잠재적인 플랫폼으로 모색되고 있습니다. 일부 전문가들은 메타버스가 우리의 생활과 업무 방식을 혁신하여 새롭고 혁신적인 방식으로 서로, 그리고 세상과 연결될 수 있는 잠재력을 가지고 있다고 믿습니다."
      },
      {
        "instruction":"양자 컴퓨팅을 간단한 용어로 설명하기",
        "input":"",
        "output":"양자 컴퓨팅은 양자역학의 원리를 사용하여 데이터에 대한 연산을 수행하는 컴퓨팅의 한 유형입니다. 양자역학은 원자 및 아원자 입자와 같은 매우 작은 입자의 거동을 다루는 물리학의 한 분야입니다.\r\n\r\n기존 컴퓨터에서는 0 또는 1을 나타낼 수 있는 비트를 사용하여 정보를 처리합니다. 반면에 양자 컴퓨터는 양자 비트 또는 큐비트를 사용하여 정보를 저장하고 처리합니다. 큐비트는 0과 1을 동시에 나타낼 수 있는데, 이를 중첩이라고 합니다. 이를 통해 양자 컴퓨터는 특정 계산을 기존 컴퓨터보다 훨씬 빠르게 수행할 수 있습니다.\r\n\r\n양자 컴퓨터는 또한 얽힘이라는 또 다른 양자 역학적 특성을 가지고 있어 특정 연산을 병렬로 수행할 수 있습니다. 즉, 동시에 많은 계산을 수행하여 속도와 성능을 더욱 높일 수 있습니다.\r\n\r\n양자 컴퓨팅은 아직 비교적 새로운 분야이며 아직 널리 사용되지는 않습니다. 그러나 양자 컴퓨팅은 암호화, 신약 개발, 머신 러닝 등 과학과 산업의 여러 분야에 혁신을 일으킬 잠재력을 가지고 있습니다."
      }
]

In [6]:
import pandas as pd
test_input_df = pd.DataFrame(test_data_input)
total_test_df = pd.concat([test_df, test_df2, test_input_df], axis = 0).reset_index().drop('index', axis = 1)
total_test_df

,instruction,input,output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...
1,스웨터의 유래는 어디에서 시작되었나요?,,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카..."
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...",,"'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로..."
...,...,...,...
95,우주왕복선에 있는 반응 제어 시스템의 내결함성을 설명하십시오.,,우주왕복선의 반응 제어 시스템(RCS)은 내결함성으로 설계되었기 때문에 구성 요소 ...
96,r미켈 셀코에 대한 랩 노래 쓰기,,"1절\r\n여러분, 잘 들어요, 할 얘기가 있어요.\r\n미켈 셀코라는 남자의 이야..."
97,최고의 아웃도어 박람회는 무엇인가요?,,세계 최고의 아웃도어 박람회에는 다음과 같은 것들이 있습니다:\r\n\r\n1. 아...
98,메타버스 설명,,"메타버스는 모든 가상 세계, 증강 현실 및 인터넷의 합계를 포함하여 가상으로 향상된..."


## KoAlpaca

In [7]:
def gen(model, prompt, user_input=None, max_new_tokens=128, temperature=0.1):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    model.eval()

    print('-' * 50)
    print(x)
    print('-' * 50)
    with torch.no_grad():
      input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
      gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        top_p = 0.95,
        top_k = 100, 
        pad_token_id = tokenizer.pad_token_id,
        eos_token_id = tokenizer.eos_token_id
      )

    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    
    return gen_text.replace(x, '')

KoAlpaca-Polyglot-5.8B

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
model_Polyglot58B = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B",
                                                         torch_dtype = torch.float16,
                                                         low_cpu_mem_usage = True).to('cuda', non_blocking = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
PROMPT_DICT = {
    "prompt_input": (
        "### 질문: {instruction}\n\n### 맥락: {input}\n\n### 답변:"
    ),
    "prompt_no_input": (
        "### 질문: {instruction}\n\n### 답변:"
    ),
}

In [12]:
def gen(model, prompt, user_input=None, max_new_tokens=128, temperature=0.1):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    model.eval()

    print('-' * 50)
    print(x)
    print('-' * 50)
    with torch.no_grad():
      input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
      gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        top_p = 0.95,
        top_k = 100, 
        pad_token_id = tokenizer.pad_token_id,
        eos_token_id = tokenizer.eos_token_id
      )

    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    
    return gen_text.replace(x, '')

In [11]:
KoAlpaca_Polyglot58B_output = []
for i in total_test_df.index:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot58B_output.append(output)

total_test_df['KoAlpaca_Polyglot5.8B_output'] = KoAlpaca_Polyglot58B_output

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변:
--------------------------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


1th output: 양파는 채소 중 하나이고, 고구마는 식물의 뿌리입니다. 양파는 둥글고 큰 양파모통이 있으며 얇은 껍질을 가지고 있습니다. 반면, 고구마는 땅 속에서 자라는 고구마처럼 생긴 덩이줄기입니다. 따라서, 양파는 뿌리이고, 고구면은 식물의 줄기입니다. 
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?
--------------------------------------------------
### 질문: 스웨터의 유래는 어디에서 시작되었나요?

### 답변:
--------------------------------------------------
2th output:  스웨터의 유래에 대해 정확히 알려진 바는 없지만, 북유럽 지역에서 추위를 막기 위해 입은 모포에서 그 유래를 찾을 수 있습니다. 이후 18세기 말 영국에서 스웨터가 처음 소개되었고, 19세기 중반에는 여성들 사이에 인기를 얻었습니다. 19세기 말에는 미국에서도 스웨터가 대중화되었습니다. 
3th instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?
--------------------------------------------------
### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:5                                                                              │
│ in gen:14                                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1565 in generate        │
│                                                                                                  │
│   1562 │   │   │   )                                                                             │
│   1563 │   │   │                                                                                 │
│   1564 │   │   │   # 13. run sample                                                              │
│ ❱ 1565 │   │   │   return self.sample(                                                           │
│   1566 │   │   │   │   input_ids,                                                                │
│   1567 │   │   │   │   logits_processor=logits_processor,                                        │
│   1568 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2612 in sample          │
│                                                                                                  │
│   2609 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2610 │   │   │                                                                                 │
│   2611 │   │   │   # forward pass to get next token                                              │
│ ❱ 2612 │   │   │   outputs = self(                                                               │
│   2613 │   │   │   │   **model_inputs,                                                           │
│   2614 │   │   │   │   return_dict=True,                                                         │
│   2615 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [10]:
# import torch
# from transformers import pipeline, AutoModelForCausalLM

# MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

# model  = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
# ).to(device=f"cuda", non_blocking=True)
# model.eval()

# pipe = pipeline(
#     'text-generation', 
#     model=model ,
#     tokenizer=MODEL,
#     device=0
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
# def ask(x, context='', is_input_full=False):
#     x = f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:"
#     ans = pipe(
#         x,
#         do_sample=True, 
#         max_new_tokens=128,
#         temperature=0.1,
#         top_p=0.95,
#         top_k = 100,
#         # return_full_text=False,
#         eos_token_id = 0,
#         no_repeat_ngram_size = 6,
#         num_return_sequences = 1
#     )
#     output = ans[0]['generated_text'].replace(x, '')
#     print(x + output)
#     # print(output)
#     # gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
#     # return gen_text.replace(x, '')

#     return output

In [11]:
# output_list = []
# for i in total_test_df.index:
#   print('-' * 50)
#   print(f'{i + 1}th')
#   output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
#   # if not total_test_df.iloc[i]['input']:
#     # output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"])
#   # else:
#     # output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
#   print('-' * 50)
#   output_list.append(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변:양파는 채소 중 하나이고, 고구마는 식물의 뿌리입니다. 양파는 둥글고 큰 양파모통의 비대한 원추형 뿌리를 가지고 있습니다. 반면 고구마는 고구마 줄기로부터 분화된 작고 길쭉한 뿌리를 가지고 있어요. 따라서 고구마는 뿌리이며, 양파는 식물의 줄기와 잎입니다. 
1th output: ### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변:양파는 채소 중 하나이고, 고구마는 식물의 뿌리입니다. 양파는 둥글고 큰 양파모통의 비대한 원추형 뿌리를 가지고 있습니다. 반면 고구마는 고구마 줄기로부터 분화된 작고 길쭉한 뿌리를 가지고 있어요. 따라서 고구마는 뿌리이며, 양파는 식물의 줄기와 잎입니다. 
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

### 질문: r미켈 셀코에 대한 랩 노래 쓰기

### 답변:미켈 셀코는 체코의 자동차 보험 세일즈맨으로, 자신의 고객이 차를 구매하게끔 설득하기 위해 다양한 광고와 이벤트를 주최합니다. 그의 완벽한 고객 만족도는 그의 서비스와 제품에 대한 평판을 높이는 데 큰 역할을 합니다.
97th output: ### 질문: r미켈 셀코에 대한 랩 노래 쓰기

### 답변:미켈 셀코는 체코의 자동차 보험 세일즈맨으로, 자신의 고객이 차를 구매하게끔 설득하기 위해 다양한 광고와 이벤트를 주최합니다. 그의 완벽한 고객 만족도는 그의 서비스와 제품에 대한 평판을 높이는 데 큰 역할을 합니다.
98th instruction: 최고의 아웃도어 박람회는 무엇인가요?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 최고의 아웃도어 박람회는 무엇인가요?

### 답변:네, 아웃도어 박람회는 다양한 등산용품, 캠핑용품, 아웃도어 장비 등을 전시하고 판매하는 행사입니다. 이를 통해 소비자들은 다양한 제품들을 비교하고 구매할 수 있습니다. 최근에는 대형 텐트, 캠핑카, 하이킹용 자전거 등의 대형 아웃도어 제품들이 인기를 끌고 있습니다. 또한, 캠핑용품 시장이 매년 높은 성장률을 보이며, 다양한 새로운 제품들이 출시되고 있습니다. 
98th output: ### 질문: 최고의 아웃도어 박람회는 무엇인가요?

### 답변:네, 아웃도어 박람회는 다양한 등산용품, 캠핑용품, 아웃도어 장비 등을 전시하고 판매하는 행사입니다. 이를 통해 소비자들은 다양한 제품들을 비교하고 구매할 수 있습니다. 최근에는 대형 텐트, 캠핑카, 하이킹용 자전거 등의 대형 아웃도어 제품들이 인기를 끌고 있습니다. 또한, 캠핑용품 시장이 매년 높은 성장률을 보이며, 다양한 새로운 제품들이 출시되고 있습니다. 
99th instruction: 메타버스 설명


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 메타버스 설명

### 답변:메타버스(Metaverse)란 3차원 이상의 가상공간을 의미하는 말로, 가상현실(VR)과 달리 3D 그래픽 등을 활용하여 현실과 흡사한 가짜 세계를 실제로 체험할 수 있는 기능을 가진 웹 기술입니다. 이는 게임에서의 상황을 예로 들자면, 가상현실에서의 게임은 '나'를 대신하는 캐릭터가 가상 공간에서 게임을 하지만, 메타버스에서의 게임은 현실에서의 실제 공간에서 실제 사람이 참여하여 게임을 즐기게 됩니다. 이러한 차이점으로 보았을 때, 메타버스는 기존의 가상현실
99th output: ### 질문: 메타버스 설명

### 답변:메타버스(Metaverse)란 3차원 이상의 가상공간을 의미하는 말로, 가상현실(VR)과 달리 3D 그래픽 등을 활용하여 현실과 흡사한 가짜 세계를 실제로 체험할 수 있는 기능을 가진 웹 기술입니다. 이는 게임에서의 상황을 예로 들자면, 가상현실에서의 게임은 '나'를 대신하는 캐릭터가 가상 공간에서 게임을 하지만, 메타버스에서의 게임은 현실에서의 실제 공간에서 실제 사람이 참여하여 게임을 즐기게 됩니다. 이러한 차이점으로 보았을 때, 메타버스는 기존의 가상현실
100th instruction: 양자 컴퓨팅을 간단한 용어로 설명하기
### 질문: 양자 컴퓨팅을 간단한 용어로 설명하기

### 답변:양자 컴퓨팅은 기존의 컴퓨터의 연산 방식과는 다른 원리로 작동되는 컴퓨팅 기술입니다. 이 기술은 양자 물리학의 원리를 기반으로 하며, 양자 컴퓨터와 양자 계산 등에 사용됩니다. 즉, 양자 컴퓨터는 양자 물리학의 원리를 활용해 문제를 해결하는 컴퓨터입니다. 
100th output: ### 질문: 양자 컴퓨팅을 간단한 용어로 설명하기

### 답변:양자 컴퓨팅은 기존의 컴퓨터의 연산 방식과는 다른 원리로 작동되는 컴퓨팅 기술입니다. 이 기술은 양자 물리학의 원리를 기반으로 하며, 양자 컴퓨터와 양자 계산 등에 사용됩니다. 즉, 양자 컴퓨터는 양자 물리학의 원리를 활용해 문제를 해결하는 컴퓨터입니다. 


In [ ]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot5.8B_output'] = output_list

In [ ]:
test_result.head()
test_result.to_excel('./test_result_koalpaca_58B.xlsx', index = False, encoding = 'utf-8')

,instruction,input,output,KoAlpaca_Polyglot5.8B_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n##...
1,스웨터의 유래는 어디에서 시작되었나요?,,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 ...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 ...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...",,"'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...","### 질문: '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것..."


8bit 양자화 모델

In [8]:
!pip install bitsandbytes xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.8 MB/s eta 0:00:00


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
model_Polyglot58B_8bit = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B",
                                                         device_map = 'auto',
                                                         load_in_8bit = True,
                                                         revision = '8bit',
                                                         )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [15]:
# import torch
# from transformers import pipeline, AutoModelForCausalLM

# MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     device_map="auto",
#     load_in_8bit=True,
#     revision="8bit",
# )
# model.eval()

# pipe = pipeline(
#     'text-generation', 
#     model=model,
#     tokenizer=MODEL,
#     # device=0
# )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [20]:
# output_list = []
# for i in total_test_df.index:
#   print('-' * 50)
#   print(f'{i + 1}th')
#   output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
#   # if not total_test_df.iloc[i]['input']:
#     # output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"])
#   # else:
#     # output = gen(model_Polyglot58B, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
#   print('-' * 50)
#   output_list.append(output)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


--------------------------------------------------
1th


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:5                                                                              │
│ in ask:3                                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_generation.py:201 in         │
│ __call__                                                                                         │
│                                                                                                  │
│   198 │   │   │   - **generated_token_ids** (`torch.Tensor` or `tf.Tensor`, present when `retu   │
│   199 │   │   │     ids of the generated text.                                                   │
│   200 │   │   """                                                                                │
│ ❱ 201 │   │   return super().__call__(text_inputs, **kwargs)                                     │
│   202 │                                                                                          │
│   203 │   def preprocess(self, prompt_text, prefix="", handle_long_generation=None, **generate   │
│   204 │   │   inputs = self.tokenizer(                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1119 in __call__          │
│                                                                                                  │
│   1116 │   │   │   │   )                                                                         │
│   1117 │   │   │   )                                                                             │
│   1118 │   │   else:                                                                             │
│ ❱ 1119 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, postproces  │
│   1120 │                                                                                         │
│   1121 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_params):   │
│   1122 │   │   return [self.run_single(item, preprocess_params, forward_params, postprocess_par  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1126 in run_single        │
│                                                                                                  │
│   1123 │                                                                                         │
│   1124 │   def run_single(self, inputs, preprocess_params, forward_params, postprocess_params):  │
│   1125 │   │   model_inputs = self.preprocess(inputs, **preprocess_params)                       │
│ ❱ 1126 │   │   model_outputs = self.forward(model_inputs, **forward_params)                      │
│   1127 │   │   outputs = self.postprocess(model_outputs, **postprocess_params)                   │
│   1128 │   │   return outputs                                                                    │
│   1129                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1025 in forward           │
│                                                                                                  │
│   1022 │   │   │   │   inference_context = self.get_inference_context()                          │
│   1023 │   │   │   │   with inference_context():                                                 │
│   1024 │   │   │   │   │   model_inputs = self._ensure_tens

In [ ]:
# test_result = total_test_df.copy()
# test_result['KoAlpaca_Polyglot5.8B_output'] = output_list

In [ ]:
# test_result.head()
# test_result.to_excel('./test_result_koalpaca_58B.xlsx', index = False, encoding = 'utf-8')

,instruction,input,output,KoAlpaca_Polyglot5.8B_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n##...
1,스웨터의 유래는 어디에서 시작되었나요?,,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 ...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 ...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...",,"'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...","### 질문: '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것..."


In [14]:
import torch

KoAlpaca_Polyglot58B_output = []
for i in total_test_df.index:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot58B_8bit, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot58B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot58B_output.append(output)

total_test_df['KoAlpaca_Polyglot5.8B_8bit_output'] = KoAlpaca_Polyglot58B_output

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변:
--------------------------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:7                                                                              │
│ in gen:14                                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1565 in generate        │
│                                                                                                  │
│   1562 │   │   │   )                                                                             │
│   1563 │   │   │                                                                                 │
│   1564 │   │   │   # 13. run sample                                                              │
│ ❱ 1565 │   │   │   return self.sample(                                                           │
│   1566 │   │   │   │   input_ids,                                                                │
│   1567 │   │   │   │   logits_processor=logits_processor,                                        │
│   1568 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2648 in sample          │
│                                                                                                  │
│   2645 │   │   │                                                                                 │
│   2646 │   │   │   # sample                                                                      │
│   2647 │   │   │   probs = nn.functional.softmax(next_token_scores, dim=-1)                      │
│ ❱ 2648 │   │   │   next_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)              │
│   2649 │   │   │                                                                                 │
│   2650 │   │   │   # finished sentences should have their next token be a padding token          │
│   2651 │   │   │   if eos_token_id is not None:                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [12]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot5.8B_8bit_output'] = KoAlpaca_Polyglot58B_output

In [13]:
test_result.head()

,instruction,input,output,KoAlpaca_Polyglot5.8B_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n##...
1,스웨터의 유래는 어디에서 시작되었나요?,,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 ...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 ...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...",,"'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...","### 질문: '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것..."


In [14]:
test_result.to_excel('./test_result_koalpaca_58B.xlsx', index = False, encoding = 'utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


KoAlpaca-Polyglot-7B

In [12]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [15]:
output_list = []
for i in total_test_df.index:
  print('-' * 50)
  print(f'{i + 1}th')
  output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
  print('-' * 50)
  output_list.append(output)

--------------------------------------------------
1th
### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양 파는 식물의 부위이며, 고구마는뿌리입니다.
--------------------------------------------------
--------------------------------------------------
2th
### 질문: 스웨터의 유래는 어디에서 시작되었나요?

### 답변: 스웜터는 1950년에 미국의 베이비 비행기 개발 팀 제프 스웜트에 의해 개발되엔 첫 비행을 했습니다.
--------------------------------------------------
--------------------------------------------------
3th
### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?

### 답변: 물체의 속도와 같은데, 빛의속도는 약 30만 킬로미터/초입니다. 따라서 토성 고리의 속성과 빛의궤절 간의 관계를 이해하기 위해서는 초당 속도의
--------------------------------------------------
--------------------------------------------------
4th
### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?
화장품 자체 제조 브랜드 런칭을 위해 OEM과 ODM용어에 대해 혼란스러움을 느끼고 있습니다

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


### 질문: 왜 초밥에는 와사비가 들어갈까요? 
초밥은 안매운 음식인데, 와사비 때문에 너무 매울 때가 있습니다. 그래서 제가 궁금한 건 초밥에 와사비가 들어가는 이유가 무엇인가요?

### 답변: 초록색 색상은 무엇의 의미를 담고 있나요?
--------------------------------------------------
--------------------------------------------------
12th
### 질문: 극장 의자 밑에는 왜 구멍이 있을까요?

### 답변: 극작가들이 극장의 밑에 구멍을 뚫어 앉아 앉아서 극장을 더욱 활기찬 것으로 설계했습니다.
--------------------------------------------------
--------------------------------------------------
13th
### 질문: 세계에서 가장 영향력 있는 인물은 누구인가요? (죽은 사람도 포함하여)

### 답변: 세곀에서 가장  영향량 있는  인물은  아메리카의 독립 전쟁 시 독려 후보인 토마스 제퍼슨입니다.
--------------------------------------------------
--------------------------------------------------
14th
### 질문: FBI 뜻은 무엇인가요? 미국 경찰인가요? 연방 사수국이라는 것은 무슨 의미인가요?

### 답변: FBI는 미국 경제부의 연방수사국(Federal Bureau of Investigation)이며, 미국 내 범죄 조사 및 수사를 수행하는 국가기관입니다.
--------------------------------------------------
--------------------------------------------------
15th
### 질문: 조선시대에 성인 나이는 몇 살이었나요? 성인식도 있었나요?

### 답변: 조선시기에는 성인 나와 성인식도가 존재했습니다.
--------

In [16]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot7B_output'] = output_list

In [17]:
test_result.head()
test_result.to_excel('./test_result_koalpaca_7B.xlsx', index = False, encoding = 'utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = "beomi/KoAlpaca"
model_Polyglot7B = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [12]:
gen(model_Polyglot7B, "점심메뉴를 추천해줘.")

--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
점심메뉴를 추천해줘.

### Response(응답):
--------------------------------------------------


'점십메뉘 추천: 샐러드'

미리 30개 정도만 테스트

In [13]:
KoAlpaca_Polyglot7B_output = []
for i in total_test_df.index[:30]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot7B, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot7B, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}\n')
  KoAlpaca_Polyglot7B_output.append(output)

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response(응답):
--------------------------------------------------
1th output: 양팔은 식물의 부위이며, 고구마는는 뿌림입니다
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
스웨터의 유래는 어디에서 시작되었나요?

### Response(응답):
--------------------------------------------------


Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 512). Running this sequence through the model will result in indexing errors


2th output: 스웨터는 1950년대 미국에서 시열 형태의 디자인 브랜드인 샤빈 스웨터와 함께 발명되엔 미국의 디저트 산업에서 확산되어 전 세계적으로 인기가 �
3th instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?

### Response(응답):
--------------------------------------------------
3th output: 토성의고리가 물체의 질량에 따라 보이는 빛의궤도와 같은게 아닙니다

In [ ]:
test_result = test_df.iloc[:30].copy()
test_result['KoAlpaca_Polyglot7B_output'] = KoAlpaca_Polyglot7B_output

In [ ]:
test_result.head()

,instruction,output,url,KoAlpaca_Polyglot7B_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...,"양팔은 식물의 부위이며, 고구마는는 뿌림입니다"
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...,스웨터는 1950년대 미국에서 시열 형태의 디자인 브랜드인 샤빈 스웨터와 함께 발명...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...,토성의고리가 물체의 질량에 따라 보이는 빛의궤도와 같은게 아닙니다. 토성의궤도는 빛...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...,OEM(Original Equipment Manufacturer)은 제품을 생산 및...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...,"'사이보그(siboggu)는 한국에서 처음으로 등장했으며, 말의 의미는 '언제나 언..."


In [ ]:
test_result.to_excel('./test_result_koalpaca7B.xlsx', index=False, encoding='utf-8')

나머지 테스트

In [17]:
KoAlpaca_Polyglot7B_output2 = []
for i in total_test_df.index[30:]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot7B, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot7B, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}\n')
  KoAlpaca_Polyglot7B_output2.append(output)

31th instruction: 건강을 유지하기 위한 세 가지 팁을 알려주세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
건강을 유지하기 위한 세 가지 팁을 알려주세요.

### Response(응답):
--------------------------------------------------
31th output: 충분한 수면, 규칙적인 운동, 건강한 식습관을 유지하세요.

32th instruction: 세 가지 기본 색은 무엇인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
세 가지 기본 색은 무엇인가요?

### Response(응답):
--------------------------------------------------
32th output: 빨강, 녹색, 파랑

33th instruction: 원자의 구조를 설명하세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately compl

In [18]:
test_result2 = total_test_df.iloc[30:].copy()
test_result2['KoAlpaca_Polyglot7B_output'] = KoAlpaca_Polyglot7B_output2

In [19]:
test_result2

,instruction,input,output,KoAlpaca_Polyglot7B_output
30,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는...","충분한 수면, 규칙적인 운동, 건강한 식습관을 유지하세요."
31,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다.","빨강, 녹색, 파랑"
32,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있...","원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성 및 중성자는 원자핵 안에 있..."
33,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기...","대기 오처리를 줄이기 위해 대중교통 이용을 증가시키거나, 자동차 사용을 줄이고, 에..."
34,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....,"어려움을 겪은 점은 친구를 둘러싸고 있는 것이었던 점입니다만, 그 친구와 함께 일하..."
...,...,...,...,...
95,우주왕복선에 있는 반응 제어 시스템의 내결함성을 설명하십시오.,,우주왕복선의 반응 제어 시스템(RCS)은 내결함성으로 설계되었기 때문에 구성 요소 ...,우주왕변선에 작동하는 반전 제어 시설은 발사 탑의 운동 방향을 조절하는 데 사용됩니...
96,r미켈 셀코에 대한 랩 노래 쓰기,,"1절\r\n여러분, 잘 들어요, 할 얘기가 있어요.\r\n미켈 셀코라는 남자의 이야...","""네 말 속에서 네가 떠오르는 랩 걸 들어주세요"""
97,최고의 아웃도어 박람회는 무엇인가요?,,세계 최고의 아웃도어 박람회에는 다음과 같은 것들이 있습니다:\r\n\r\n1. 아...,최고의 '아웃도아' 박람 회는 없습니다.
98,메타버스 설명,,"메타버스는 모든 가상 세계, 증강 현실 및 인터넷의 합계를 포함하여 가상으로 향상된...","메타분 설명은 두 개의 매우 다른 분야를 연결하는 것으로, 이를 통해 더 많은 관심..."


In [ ]:
test_result2.to_excel('./test_result_koalpaca7B2.xlsx', index=False, encoding='utf-8')

In [ ]:
output_list = []
for i in total_test_df.index:
  print('-' * 50)
  print(f'{i + 1}th')
  output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
  print('-' * 50)
  output_list.append(output)

In [ ]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot7B_output'] = output_list

In [ ]:
test_result.head()
test_result.to_excel('./test_result_koalpaca_7B.xlsx', index = False, encoding = 'utf-8')

KoAlpaca-Polyglot-12.8B 8bit 양자화모델

In [14]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 7.8 MB/s eta 0:00:00


In [15]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-12.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
    load_in_8bit=True,
    revision="8bit",
)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    # device=0
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [16]:
output_list = []
for i in total_test_df.index:
  print('-' * 50)
  print(f'{i + 1}th')
  output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
  print('-' * 50)
  output_list.append(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--------------------------------------------------
1th


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:5                                                                              │
│ in ask:3                                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_generation.py:201 in         │
│ __call__                                                                                         │
│                                                                                                  │
│   198 │   │   │   - **generated_token_ids** (`torch.Tensor` or `tf.Tensor`, present when `retu   │
│   199 │   │   │     ids of the generated text.                                                   │
│   200 │   │   """                                                                                │
│ ❱ 201 │   │   return super().__call__(text_inputs, **kwargs)                                     │
│   202 │                                                                                          │
│   203 │   def preprocess(self, prompt_text, prefix="", handle_long_generation=None, **generate   │
│   204 │   │   inputs = self.tokenizer(                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1119 in __call__          │
│                                                                                                  │
│   1116 │   │   │   │   )                                                                         │
│   1117 │   │   │   )                                                                             │
│   1118 │   │   else:                                                                             │
│ ❱ 1119 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, postproces  │
│   1120 │                                                                                         │
│   1121 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_params):   │
│   1122 │   │   return [self.run_single(item, preprocess_params, forward_params, postprocess_par  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1126 in run_single        │
│                                                                                                  │
│   1123 │                                                                                         │
│   1124 │   def run_single(self, inputs, preprocess_params, forward_params, postprocess_params):  │
│   1125 │   │   model_inputs = self.preprocess(inputs, **preprocess_params)                       │
│ ❱ 1126 │   │   model_outputs = self.forward(model_inputs, **forward_params)                      │
│   1127 │   │   outputs = self.postprocess(model_outputs, **postprocess_params)                   │
│   1128 │   │   return outputs                                                                    │
│   1129                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1025 in forward           │
│                                                                                                  │
│   1022 │   │   │   │   inference_context = self.get_inference_context()                          │
│   1023 │   │   │   │   with inference_context():                                                 │
│   1024 │   │   │   │   │   model_inputs = self._ensure_tens

In [ ]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot12B_output'] = output_list

In [ ]:
test_result.head()
test_result.to_excel('./test_result_koalpaca_12B.xlsx', index = False, encoding = 'utf-8')

In [ ]:
def ask(x, context='', is_input_full=False): # max token = 512
    x = f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:"
    ans = pipe(
        x,
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.1,
        top_p=0.95,
        top_k = 100,
        # return_full_text=False,
        eos_token_id=2,
        no_repeat_ngram_size = 6,
        num_return_sequences = 1
    )
    output = ans[0]['generated_text'].replace(x, '')
    print(x + output)
    # print(output)
    # gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    # return gen_text.replace(x, '')

    return output

In [ ]:
output_list = []
for i in total_test_df.index:
  print('-' * 50)
  print(f'{i + 1}th')
  output = ask(total_test_df.iloc[i]['instruction'], context = total_test_df.iloc[i]['input'])
  print('-' * 50)
  output_list.append(output)

In [ ]:
test_result = total_test_df.copy()
test_result['KoAlpaca_Polyglot12B_output'] = output_list

In [ ]:
test_result.head()
test_result.to_excel('./test_result_koalpaca_12B_2.xlsx', index = False, encoding = 'utf-8')

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = "beomi/KoAlpaca-Polyglot-12.8B"
model_Polyglot12B_8bit = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
    load_in_8bit=True,
    revision="8bit",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [16]:
gen(model_Polyglot12B_8bit, "점심메뉴를 추천해줘.")

--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
점심메뉴를 추천해줘.

### Response(응답):
--------------------------------------------------


'\n오늘 점심에는 멕시칸 푸드 어떠세요?'

In [13]:
gen(model_Polyglot12B_8bit, "python uptime을 찾는 코드")

--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
python uptime을 찾는 코드

### Response(응답):
--------------------------------------------------


'\nprint("python used to last")'

max_new_tokens = 128

In [14]:
KoAlpaca_Polyglot12B_output = []
for i in total_test_df.index[:30]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot12B_output.append(output)

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response(응답):
--------------------------------------------------
1th output: 
양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
스웨터의 유래는 어디에서 시작되었나요?

### Response(응답):
--------------------------------------------------
2th output: 
스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
3th instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 

In [15]:
test_result = total_test_df.iloc[:30].copy()
test_result['KoAlpaca_Polyglot12B_8bit_128_output'] = KoAlpaca_Polyglot12B_output

In [16]:
test_result.head()

,instruction,output,url,KoAlpaca_Polyglot12B_8bit_128_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n토성의 고리와 같은 천체에서 빛의 띠로 보이거나 장노출 사진에서 빛의 궤적이...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...,\n제조판매업자와 제조업자 각각 OEM과 ODM을 이용하여 제품을 생산합니다. OE...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n'사이언티스트(scientist)'와 '유기체(organism)'의 합성어인 '...


In [20]:
test_result.to_csv('./test_result_koalpaca12B_8bit_128.csv', index = False, encoding = 'utf-8')

나머지 테스트

In [17]:
KoAlpaca_Polyglot12B_output2 = []
for i in total_test_df.index[30:]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}\n')
  KoAlpaca_Polyglot12B_output2.append(output)

31th instruction: 건강을 유지하기 위한 세 가지 팁을 알려주세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
건강을 유지하기 위한 세 가지 팁을 알려주세요.

### Response(응답):
--------------------------------------------------
31th output: 
1. 규칙적인 운동을 하세요.
2. 건강한 식습관을 유지하세요.
3. 충분한 수면을 취하세요.

32th instruction: 세 가지 기본 색은 무엇인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
세 가지 기본 색은 무엇인가요?

### Response(응답):
--------------------------------------------------
32th output: 
세 가지 색은 빨강, 녹색, 파랑입니다.

33th instruction: 원자의 구조를 설명하세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a

In [18]:
test_result2 = total_test_df.iloc[30:].copy()
test_result2['KoAlpaca_Polyglot12B_8bit_128_output'] = KoAlpaca_Polyglot12B_output2

In [19]:
test_result2

,instruction,input,output,KoAlpaca_Polyglot12B_8bit_128_output
30,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는...",\n1. 규칙적인 운동을 하세요.\n2. 건강한 식습관을 유지하세요.\n3. 충분한...
31,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다.","\n세 가지 색은 빨강, 녹색, 파랑입니다."
32,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있...","\n원자의 전자는 원자핵 주위를 회전하는데, 전자의 갯수와 각 전자의 회전 방향에 ..."
33,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기...",\n대기 오염은 다양한 방법을 통해 줄일 수 있습니다. 가장 중요한 것은 대기 오염...
34,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....,\n과거에 어려운 결정을 내려야 했던 경험이 있습니다.
...,...,...,...,...
95,우주왕복선에 있는 반응 제어 시스템의 내결함성을 설명하십시오.,,우주왕복선의 반응 제어 시스템(RCS)은 내결함성으로 설계되었기 때문에 구성 요소 ...,\n내결함성은 제어 시스템이 예상대로 작동하지 않을 경우 이를 수정하는 능력을 의미...
96,r미켈 셀코에 대한 랩 노래 쓰기,,"1절\r\n여러분, 잘 들어요, 할 얘기가 있어요.\r\n미켈 셀코라는 남자의 이야...","\n""제가 쓸게요,\n미켈 셀코에 대하여 랩 노래를 쓰겠습니다.\n그는 미국에서 태..."
97,최고의 아웃도어 박람회는 무엇인가요?,,세계 최고의 아웃도어 박람회에는 다음과 같은 것들이 있습니다:\r\n\r\n1. 아...,"""최고의 아웃도어 박람회는 보통 National Airport에서 열리는 국내 아..."
98,메타버스 설명,,"메타버스는 모든 가상 세계, 증강 현실 및 인터넷의 합계를 포함하여 가상으로 향상된...","\n메타버스로 설명하자면 아바타를 통해 가상 세계에서 이루어지는 활동을 말하며, 현..."


In [20]:
test_result2.to_excel('./test_result_koalpaca12B_2.xlsx', index=False, encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


max_new_tokens = 256

In [21]:
KoAlpaca_Polyglot12B_256_output = []
for i in test_df.index[:30]:
  print(f'{i + 1}th instruction: {test_df.iloc[i]["instruction"]}')
  if not test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, test_df.iloc[i]["instruction"], max_new_tokens = 256)
  else:
    output = gen(model_Polyglot58B, test_df.iloc[i]["instruction"], test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot12B_256_output.append(output)

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response(응답):
--------------------------------------------------
1th output: 
양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
스웨터의 유래는 어디에서 시작되었나요?

### Response(응답):
--------------------------------------------------
2th output: 
스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
3th instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 

In [22]:
test_result['KoAlpaca_Polyglot12B_8bit_256_output'] = KoAlpaca_Polyglot12B_256_output

In [23]:
test_result.head()

,instruction,output,url,KoAlpaca_Polyglot12B_8bit_128_output,KoAlpaca_Polyglot12B_8bit_256_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.,\n양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.,\n스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n토성의 고리와 같은 천체에서 빛의 띠로 보이거나 장노출 사진에서 빛의 궤적이...,\n토성의 고리와 같은 천체에서 빛의 띠로 보이거나 장노출 사진에서 빛의 궤적이...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...,\n제조판매업자와 제조업자 각각 OEM과 ODM을 이용하여 제품을 생산합니다. OE...,\n제조판매업자와 제조업자 각각 OEM과 ODM을 이용하여 제품을 생산합니다. OE...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n'사이언티스트(scientist)'와 '유기체(organism)'의 합성어인 '...,\n'사이언티스트(scientist)'와 '유기체(organism)'의 합성어인 '...


In [ ]:
test_result.to_excel('./test_result_koalpaca12B_8bit.xlsx', index=False, encoding='utf-8')

test input data

In [17]:
KoAlpaca_Polyglot12B_output = []
for i in test_input_df.index:
  print(f'{i + 1}th instruction: {test_input_df.iloc[i]["instruction"]}')
  if not test_input_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, test_input_df.iloc[i]["instruction"], max_new_tokens = 256)
  else:
    output = gen(model_Polyglot12B_8bit, test_input_df.iloc[i]["instruction"], user_input = test_input_df.iloc[i]['input'], max_new_tokens = 256)
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot12B_output.append(output)

1th instruction: 다음 문장을 참 또는 거짓으로 분류하십시오.
--------------------------------------------------
Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
다음 문장을 참 또는 거짓으로 분류하십시오.

### Input(입력):
대법원은 미국 최고법원입니다.

### Response(응답):
--------------------------------------------------
1th output: 
'참'입니다.
2th instruction: 아래 문장의 각 단어에 품사를 지정하십시오.
--------------------------------------------------
Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
아래 문장의 각 단어에 품사를 지정하십시오.

### Input(입력):
새끼 고양이는 종종 신나게 뛰어다니기도 합니다.

### Response(응답):
--------------------------------------------------
2th output: 
단어: 

In [18]:
test_input_df['KoAlpaca_Polyglot12B_8bit_output'] = KoAlpaca_Polyglot12B_output

In [19]:
test_input_df.head()

,instruction,input,output,KoAlpaca_Polyglot12B_8bit_output
0,다음 문장을 참 또는 거짓으로 분류하십시오.,대법원은 미국 최고법원입니다.,참.,\n'참'입니다.
1,아래 문장의 각 단어에 품사를 지정하십시오.,새끼 고양이는 종종 신나게 뛰어다니기도 합니다.,새끼(Noun) 고양이(Noun) 는 은 종종 Adverb 신나게 Adverb 뛰어...,"\n단어: 새끼, 고양이, 종종, 뛰어다니기, 새끼고양이는, 종종, 신나게, 뛰어다..."
2,이 텍스트의 주요 아이디어는 무엇인가요?,로봇 산업은 오늘날 경제에서 가장 빠르게 성장하는 분야 중 하나가 되었습니다. 로보...,"로봇 공학은 빠르게 성장하는 분야이며, 제조, 의료, 국방 등 다양한 산업과 애플리...",\n로봇 산업이 경제에서 가장 빠르게 발전하는 분야 중 하나이며 다양한 산업과 애플...
3,화씨에서 섭씨로 온도를 변환합니다.,94F,입력받은 화씨 94도는 섭씨 34.44˚입니다.,\nFahrenheit temperature is switched from Cels...
4,다음 문장을 단수형에서 복수형으로 바꾸세요.,고양이가 술을 마시고 있습니다.,고양이들이 술을 마시고 있습니다.,\nThe cats are drinking.


In [ ]:
test_input_df.to_excel('./test_result_koalpaca12B_8bit_new_data.xlsx', index=False, encoding='utf-8')

나머지 테스트

In [21]:
KoAlpaca_Polyglot12B_output2 = []
for i in total_test_df.index[30:80]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], max_new_tokens = 256)
  else:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'], max_new_tokens = 256)
  print(f'{i + 1}th output: {output}\n')
  KoAlpaca_Polyglot12B_output2.append(output)

31th instruction: 건강을 유지하기 위한 세 가지 팁을 알려주세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
건강을 유지하기 위한 세 가지 팁을 알려주세요.

### Response(응답):
--------------------------------------------------
31th output: 
1. 규칙적인 운동을 하세요.
2. 건강한 식습관을 유지하세요.
3. 충분한 수면을 취하세요.

32th instruction: 세 가지 기본 색은 무엇인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
세 가지 기본 색은 무엇인가요?

### Response(응답):
--------------------------------------------------
32th output: 
세 가지 색은 빨강, 녹색, 파랑입니다.

33th instruction: 원자의 구조를 설명하세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a

In [22]:
test_result2 = total_test_df.iloc[30:80].copy()
test_result2['KoAlpaca_Polyglot12B_8bit_256_output'] = KoAlpaca_Polyglot12B_output2

In [23]:
test_result2

,instruction,input,output,KoAlpaca_Polyglot12B_8bit_256_output
30,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는...",\n1. 규칙적인 운동을 하세요.\n2. 건강한 식습관을 유지하세요.\n3. 충분한...
31,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다.","\n세 가지 색은 빨강, 녹색, 파랑입니다."
32,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있...","\n원자의 전자는 원자핵 주위를 회전하는데, 전자의 갯수와 각 전자의 회전 방향에 ..."
33,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기...",\n대기 오염은 다양한 방법으로 줄일 수 있습니다. 가장 중요한 것은 대기 오염의 ...
34,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....,\n과거에 어려운 결정을 내려야 했던 경험이 있습니다.
35,홀수 중 하나를 밝히세요.,"트위터, 인스타그램, 텔레그램",텔레그램입니다.,"\n홀수는 \n2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,..."
36,다음 분수가 1/4에 해당하는 이유를 설명하세요.,4/16,분모와 분자를 모두 4로 나눌 수 있기 때문에 1/4에 해당합니다.,4/16을 1/4로 나누면 몫이 4이고 나머지가 없습니다.
37,중요한 진로 결정을 내려야 하는 주인공의 이야기를 3인칭 내레이션으로 단편 소설을 ...,,"안나는 여러 대학교의 입학 가능성이 있었으나, 제발스러운 직업들을 향해 유혹 당하는...",\n나는 진로를 결정하는 것이 매우 어려운 일이라는 것을 알고 있습니다. 하지만 나...
38,집의 3D 모델을 렌더링합니다.,,집의 3D 모델링 결과물 입니다. (이미지 첨부),\n작업이 완료되었음을 알려드립니다.
39,이 문장에 철자와 문법 오류가 있는지 평가하세요.,그는 식사를 마치고 식당을 나섰습니다.,그 문장에는 철자나 문법에 대한 오류가 없습니다.,\n그는 식당에서 식사를 마치고 식당에서 나왔습니다.


In [ ]:
test_result2.to_excel('./test_result_koalpaca12B_256.xlsx', index=False, encoding='utf-8')

KoVicuna_7b

In [ ]:
!pip install bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL = "junelee/ko_vicuna_7b"
model_kovicuna7B = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype = torch.float16,
).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [ ]:
gen(model_Polyglot12B_8bit, "점심메뉴를 추천해줘.")

--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
점심메뉴를 추천해줘.

### Response(응답):
--------------------------------------------------


'\n오늘 점심에는 멕시칸 푸드 어떠세요?'

In [ ]:
gen(model_Polyglot12B_8bit, "python uptime을 찾는 코드")

--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
python uptime을 찾는 코드

### Response(응답):
--------------------------------------------------


'\nprint("python used to last")'

max_new_tokens = 128

In [ ]:
KoAlpaca_Polyglot12B_output = []
for i in total_test_df.index[:30]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}')
  KoAlpaca_Polyglot12B_output.append(output)

1th instruction: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response(응답):
--------------------------------------------------
1th output: 
양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
2th instruction: 스웨터의 유래는 어디에서 시작되었나요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
스웨터의 유래는 어디에서 시작되었나요?

### Response(응답):
--------------------------------------------------
2th output: 
스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
3th instruction: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 

In [ ]:
test_result = total_test_df.iloc[:30].copy()
test_result['KoAlpaca_Polyglot12B_8bit_128_output'] = KoAlpaca_Polyglot12B_output

In [ ]:
test_result.head()

,instruction,output,url,KoAlpaca_Polyglot12B_8bit_128_output
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n양파와 고구마는 모두 구근이 아닌 뿌리채소입니다.
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...,\n스웨터는 기원전 약 8세기 고대 그리스와 로마에서 시작되었습니다.
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n토성의 고리와 같은 천체에서 빛의 띠로 보이거나 장노출 사진에서 빛의 궤적이...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...,\n제조판매업자와 제조업자 각각 OEM과 ODM을 이용하여 제품을 생산합니다. OE...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...,\n'사이언티스트(scientist)'와 '유기체(organism)'의 합성어인 '...


In [ ]:
test_result.to_csv('./test_result_koalpaca12B_8bit_128.csv', index = False, encoding = 'utf-8')

나머지 테스트

In [ ]:
KoAlpaca_Polyglot12B_output2 = []
for i in total_test_df.index[30:]:
  print(f'{i + 1}th instruction: {total_test_df.iloc[i]["instruction"]}')
  if not total_test_df.iloc[i]['input']:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"])
  else:
    output = gen(model_Polyglot12B_8bit, total_test_df.iloc[i]["instruction"], total_test_df.iloc[i]['input'])
  print(f'{i + 1}th output: {output}\n')
  KoAlpaca_Polyglot12B_output2.append(output)

31th instruction: 건강을 유지하기 위한 세 가지 팁을 알려주세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
건강을 유지하기 위한 세 가지 팁을 알려주세요.

### Response(응답):
--------------------------------------------------
31th output: 
1. 규칙적인 운동을 하세요.
2. 건강한 식습관을 유지하세요.
3. 충분한 수면을 취하세요.

32th instruction: 세 가지 기본 색은 무엇인가요?
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a response that appropriately completes the request.
명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
세 가지 기본 색은 무엇인가요?

### Response(응답):
--------------------------------------------------
32th output: 
세 가지 색은 빨강, 녹색, 파랑입니다.

33th instruction: 원자의 구조를 설명하세요.
--------------------------------------------------
Below is an instruction that describes a task.
아래는 작업을 설명하는 명령어입니다.

Write a

In [ ]:
test_result2 = total_test_df.iloc[30:].copy()
test_result2['KoAlpaca_Polyglot12B_8bit_128_output'] = KoAlpaca_Polyglot12B_output2

In [ ]:
test_result2

,instruction,input,output,KoAlpaca_Polyglot12B_8bit_128_output
30,건강을 유지하기 위한 세 가지 팁을 알려주세요.,,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는...",\n1. 규칙적인 운동을 하세요.\n2. 건강한 식습관을 유지하세요.\n3. 충분한...
31,세 가지 기본 색은 무엇인가요?,,"기본 색은 빨강, 파랑, 노랑입니다.","\n세 가지 색은 빨강, 녹색, 파랑입니다."
32,원자의 구조를 설명하세요.,,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있...","\n원자의 전자는 원자핵 주위를 회전하는데, 전자의 갯수와 각 전자의 회전 방향에 ..."
33,대기 오염을 어떻게 줄일 수 있나요?,,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기...",\n대기 오염은 다양한 방법을 통해 줄일 수 있습니다. 가장 중요한 것은 대기 오염...
34,어려운 결정을 내려야 했던 때를 설명하세요.,,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....,\n과거에 어려운 결정을 내려야 했던 경험이 있습니다.
...,...,...,...,...
95,우주왕복선에 있는 반응 제어 시스템의 내결함성을 설명하십시오.,,우주왕복선의 반응 제어 시스템(RCS)은 내결함성으로 설계되었기 때문에 구성 요소 ...,\n내결함성은 제어 시스템이 예상대로 작동하지 않을 경우 이를 수정하는 능력을 의미...
96,r미켈 셀코에 대한 랩 노래 쓰기,,"1절\r\n여러분, 잘 들어요, 할 얘기가 있어요.\r\n미켈 셀코라는 남자의 이야...","\n""제가 쓸게요,\n미켈 셀코에 대하여 랩 노래를 쓰겠습니다.\n그는 미국에서 태..."
97,최고의 아웃도어 박람회는 무엇인가요?,,세계 최고의 아웃도어 박람회에는 다음과 같은 것들이 있습니다:\r\n\r\n1. 아...,"""최고의 아웃도어 박람회는 보통 National Airport에서 열리는 국내 아..."
98,메타버스 설명,,"메타버스는 모든 가상 세계, 증강 현실 및 인터넷의 합계를 포함하여 가상으로 향상된...","\n메타버스로 설명하자면 아바타를 통해 가상 세계에서 이루어지는 활동을 말하며, 현..."


In [ ]:
test_result2.to_excel('./test_result_koalpaca12B_2.xlsx', index=False, encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


## Fine tuning